# Reddit r/WritingPrompts data downloader

In [4]:
# Step 2: Import libraries
import praw
import pandas as pd
import time
import os

In [5]:
# Step 3: Set up Reddit API credentials
reddit = praw.Reddit(
    client_id='-eqmyJxGZcoKmesk8lZzdQ',
    client_secret='tDsl-SGg5_ZSNSmoHtlvTwGwxkjSrw',
    user_agent='-eqmyJxGZcoKmesk8lZzdQ'
)

In [6]:
# Step 4: Choose a subreddit and fetch posts and comments
subreddit_name = "WritingPrompts"
post_limit = 1000  # Number of posts to scrape

subreddit = reddit.subreddit(subreddit_name)

posts_data = []
comments_data = []

try:
    for index, post in enumerate(subreddit.hot(limit=post_limit)):
        if post.title[:4] != '[WP]':
            continue

        posts_data.append({
            'title': post.title,
            'id': post.id,
            'num_comments': post.num_comments
        })

        try:
            post.comments.replace_more(limit=0)
            for comment in post.comments.list():
                if comment.score < 5 or len(comment.body) < 256:
                    continue
                comments_data.append({
                    'post_id': post.id,
                    'comment_id': comment.id,
                    'comment_body': comment.body,
                    'comment_score': comment.score
                })

        except Exception as e:
            print(f"Error processing comments for post {post.id}: {e}")


except Exception as e:
    print(f"An unexpected error occurred: {e}")
    time.sleep(5)

# Create DataFrames
posts_df = pd.DataFrame(posts_data)
comments_df = pd.DataFrame(comments_data)

In [7]:
print(len(posts_df))
posts_df.head()

838


,title,id,num_comments
0,[WP] The curse was broken. You remember who yo...,1lx0sij,8
1,[WP] For the first time in all your years as a...,1lwvgcx,11
2,[WP]The Witch Of Nowhere was banned to live in...,1lwypks,11
3,"[WP] When you were a child, you had a very str...",1lx52x2,3
4,[WP] People laugh at your friend for his super...,1lx6u9l,2


In [8]:
print(len(comments_df))
comments_df.head()

751


,post_id,comment_id,comment_body,comment_score
0,1lx0sij,n2ij42j,"I watch, as my wife sleeps, with our newborn d...",44
1,1lx0sij,n2is0gm,It was all part of the plan. Go in without mem...,14
2,1lx0sij,n2ij67i,She was the mother of all things... the creato...,9
3,1lwvgcx,n2hlpid,"You learn a lot in 3,000 years of granting wis...",197
4,1lwvgcx,n2hkjsi,"""What do you mean ""wish denied""?"" the current ...",55


In [9]:
# Step 6: Save the data
posts_df.to_csv("../data/raw/reddit_posts_11072025.csv", index=False)
comments_df.to_csv("../data/raw/reddit_comments_11072025.csv", index=False)

print("Data saved to reddit_posts.csv and reddit_comments.csv")

Data saved to reddit_posts.csv and reddit_comments.csv


In [10]:
# Have to finish pretraining the model today and trying it out. for sure.
# Lets try and split into training and validation sets
training_df = pd.DataFrame(columns=['Post_ID', 'Comment_ID', 'Prompt', 'Story', 'Story_Score'])
validation_df = pd.DataFrame(pd.DataFrame(columns=['Post_ID', 'Prompt']))

for index, post in posts_df.iterrows():
    if post.id in comments_df.post_id.values:
        comments = comments_df[comments_df.post_id == post.id]
        for index, comment in comments.iterrows():
            training_value = {
                'Post_ID': post.id,
                'Comment_ID': comment.comment_id,
                'Prompt': post.title[4:],  # Remove '[WP]' from the title
                'Story': comment.comment_body,
                'Story_Score': comment.comment_score
            }
            training_df = pd.concat([training_df, pd.DataFrame([training_value])], ignore_index=True)
    else:
        validation_value = {
            'Post_ID': post.id,
            'Prompt': post.title
        }
        validation_df = pd.concat([validation_df, pd.DataFrame([validation_value])], ignore_index=True) 

In [11]:
print(len(training_df))
training_df.head()

751


,Post_ID,Comment_ID,Prompt,Story,Story_Score
0,1lx0sij,n2ij42j,The curse was broken. You remember who you we...,"I watch, as my wife sleeps, with our newborn d...",44
1,1lx0sij,n2is0gm,The curse was broken. You remember who you we...,It was all part of the plan. Go in without mem...,14
2,1lx0sij,n2ij67i,The curse was broken. You remember who you we...,She was the mother of all things... the creato...,9
3,1lwvgcx,n2hlpid,For the first time in all your years as a Gen...,"You learn a lot in 3,000 years of granting wis...",197
4,1lwvgcx,n2hkjsi,For the first time in all your years as a Gen...,"""What do you mean ""wish denied""?"" the current ...",55


In [12]:
print(len(validation_df))
validation_df.head()

570


,Post_ID,Prompt
0,1lx52x2,"[WP] When you were a child, you had a very str..."
1,1lx6u9l,[WP] People laugh at your friend for his super...
2,1lx1rv5,"[WP] Long ago, a dragon saved a human king fro..."
3,1lx7l9c,"[WP] ""so you where a paladin what did you have..."
4,1lx3vm0,[WP] You’re an immortal and a drunk. You’ve so...
